## Import

In [12]:
import copy
import json
import time
import jieba
import string
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from thefuzz import fuzz
from rank_bm25 import BM25Okapi

from sklearn.metrics import accuracy_score

In [13]:
import openai
openai.api_key = 'sk-xxxxx'

In [14]:
instruction = '''以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'''
instruction

'以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'

## Data

In [14]:
with open('../data/list_dict_test_2022.json', 'r', encoding="utf-8") as f:
    list_dict_test = json.load(f)

In [15]:
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    question, answer = dict_test['Question'], dict_test['Answer']
    a, b, c, d, e = dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']
    question = question.replace("（ ）。", "")
    input = instruction + f"问题：{question}: (A){a}, (B){b}, (C){c}, (D){d}, (E){e}\n" + "答案："
    dict_test['Input'] = input
    # print(input, '\t' ,answer)

100%|██████████| 494/494 [00:00<00:00, 493447.53it/s]


## Test

In [16]:
def get_response_dict(dict_test):
    input = dict_test['Input']
    # print(input)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        # max_tokens = 1,
        messages=[
            # {"role": "system", "content": instruction_system}, 
            {"role": "user", "content": input}
        ],
    )
    print(f'Response: {response["choices"][0]["message"]["content"]}\n', )
    print(f"True Label: {dict_test['Answer']}\n")
    print(f"Finish_reason: {response['choices'][0]['finish_reason']}\n")
    print(response['usage']._previous)
    return response

In [17]:
idx = 255
dict_test = list_dict_test[idx]
print(dict_test['Input'])

以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项
问题：关于低位肠梗阻正确的是: (A)呕吐症状出现早, (B)梗阻部位多为空肠, (C)腹部X线可见多个阶梯状气液平, (D)腹痛腹胀不明显, (E)呕吐物多为胃内容物
答案：


In [18]:
response = get_response_dict(dict_test)

Response: C

True Label: C

Finish_reason: stop

{'prompt_tokens': 154, 'completion_tokens': 1, 'total_tokens': 155}


## Task

In [15]:
name_task = "Prompt"

In [ ]:
with open(f'../input/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

## Run

In [21]:
!python turbo_json.py Prompt

name_task:Prompt


 26%|██▌       | 126/494 [00:20<00:12, 29.12it/s]




ready for 494 data

Wrong: 147

Wrong: 448

messages length:494, num_threads:64, time cost:339.1046471595764

Supplement 2 failed

Get response:494
Cost: 1.32 RMB



100%|██████████| 494/494 [00:25<00:00, 19.02it/s]

100%|██████████| 2/2 [00:01<00:00,  1.28it/s]


## Performance

### Read

In [37]:
with open(f'../log/{name_task}_0_494.json', 'r', encoding="utf-8") as f:
    list_dict_test = json.load(f)

In [38]:
len(list_dict_test), len([dict_test for dict_test in list_dict_test if dict_test.get('usage')])

(494, 494)

### Supplement

In [24]:
# # list_dict_result_failed = [ dict_test for dict_test in list_dict_test if not dict_test.get('usage') ]
# list_dict_result_failed = [list_dict_test[5]]
# print(f"{len(list_dict_result_failed)} failed")
# for dict_test in list_dict_result_failed:
#     input = dict_test['Input']
#     id = dict_test['ID']
#     response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo", 
#             temperature=0,
#             n=1, 
#             top_p=1,
#             frequency_penalty=0,
#             presence_penalty=0,
#             messages=[
#                 # {"role": "system", "content": 'instruction for system'}, 
#                 {"role": "user", "content": input}]
#     )
#     result = response["choices"][0]["message"]["content"]
#     dict_test['Result'] = result
#     # usage
#     dict_test['usage'] = response['usage']._previous
#     # save_log(dict_test)
#     print(input, result, sep='\n')
print('ok')

ok


In [25]:
dict_test

{'ID': 255,
 'Question': '关于低位肠梗阻正确的是',
 'A': '呕吐症状出现早',
 'B': '梗阻部位多为空肠',
 'C': '腹部X线可见多个阶梯状气液平',
 'D': '腹痛腹胀不明显',
 'E': '呕吐物多为胃内容物',
 'Answer': 'C',
 'Analysis': '低位肠梗阻，扩张的肠袢在腹中部，腹部X线呈“阶梯状”排列，可见多个气液平。',
 'Input': '以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n问题：关于低位肠梗阻正确的是: (A)呕吐症状出现早, (B)梗阻部位多为空肠, (C)腹部X线可见多个阶梯状气液平, (D)腹痛腹胀不明显, (E)呕吐物多为胃内容物\n答案：'}

In [ ]:
# with open(f'log/{name_task}_0_494.json', 'w', encoding="utf-8") as f:
#     f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

### Prediction

In [27]:
def get_prediction(result_pred, list_option_name):
    list_str_split = ['最终答案是','最终答案为','最可能的选项是','最可能的诊断是','正确的选项是', '正确选项是', '正确选项为','最可能的答案是','最有可能的答案是']
    for str_split in list_str_split:
        if len(result_pred.split(str_split))>1:
            result_pred = result_pred.split(str_split)[-1]
            result_pred = result_pred.split('。')[0]
            break
    list_option_idx = [ chr(ord('A') + idx) for idx in range(len(list_option_name))]
    list_option_idx_name = [ [option_idx, option_name] for option_idx, option_name in zip(list_option_idx, list_option_name)]
    list_option_idx_name = sorted(list_option_idx_name, key=lambda x: len(x[1]), reverse=True)
    list_option_match = []
    # search option_idx and option_name
    for option_idx, option_name in list_option_idx_name:
        if option_idx in result_pred or option_name in result_pred:
            list_option_match.append(option_idx)
            if option_name in result_pred:
                result_pred = result_pred.replace(option_name, '')
    if len(list_option_match)==1:
        return list_option_match[0]
    elif len(list_option_match)>1:
        return None
    else:
        return None

In [28]:
list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    list_option_name = [dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']]
    dict_test['Prediction'] = get_prediction(dict_test['Result'], list_option_name)
    if not dict_test['Prediction']:
        list_dict_result_none.append(dict_test)
    elif dict_test['Answer'] == dict_test['Prediction']:
        list_dict_result_correct.append(dict_test)
    else:
        list_dict_result_wrong.append(dict_test)
    # if len(dict_test['Result']) > 6:
    #     print(dict_test['Result']) 

100%|██████████| 494/494 [00:00<00:00, 98877.89it/s]


In [29]:
count_correct = len(list_dict_result_correct)
count_wrong = len(list_dict_result_wrong)
count_none = len(list_dict_result_none)
acc = round(len(list_dict_result_correct) / len(list_dict_test)*100, 2)
print(f"Accuracy is {acc}% with {count_correct} correct, {count_wrong} wrong, and {count_none} none")

Accuracy is 50.0% with 247 correct, 241 wrong, and 6 none


In [30]:
# Too long 111

In [31]:
with open(f'../log/{name_task}_0_494.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

### Final

In [16]:
with open(f'../log/{name_task}_0_494.json', 'r', encoding="utf-8") as f:
    list_dict_test_sample = json.load(f)
with open('../data/list_test_knowledge_idx.txt', 'r') as f:
    list_test_knowledge_idx = f.readlines()
with open('../data/list_test_example_idx.txt', 'r') as f:
    list_test_example_idx = f.readlines()
list_test_knowledge_idx = [ int(idx.strip()) for idx in list_test_knowledge_idx if idx.strip() ]
list_test_example_idx = [ int(idx.strip()) for idx in list_test_example_idx if idx.strip() ]
list_dict_test_knowledge = [ list_dict_test_sample[idx] for idx in list_test_knowledge_idx ]
list_dict_test_example = [ list_dict_test_sample[idx] for idx in list_test_example_idx ]
print(len(list_dict_test_sample), len(list_dict_test_knowledge), len(list_dict_test_example))

494 181 313


In [17]:
def cal_performance(list_dict_test_sample):
    list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
    for idx, dict_test in enumerate(list_dict_test_sample):
        if not dict_test['Prediction']:
            list_dict_result_none.append(dict_test)
        elif dict_test['Answer'] == dict_test['Prediction']:
            list_dict_result_correct.append(dict_test)
        else:
            list_dict_result_wrong.append(dict_test)
    count_correct = len(list_dict_result_correct)
    count_wrong = len(list_dict_result_wrong)
    count_none = len(list_dict_result_none)
    acc = len(list_dict_result_correct) / len(list_dict_test_sample)
    print(f"Accuracy is {round(acc*100, 2)}% with {count_none} none, {count_correct} correct, and {count_wrong} wrong， total {len(list_dict_test_sample)}")
    return round(acc, 4), count_none, count_correct, count_wrong

In [18]:
acc_MK, count_none_MK, _, _ = cal_performance(list_dict_test_knowledge)
acc_CA, count_none_CA, _, _ = cal_performance(list_dict_test_example)
acc_all, count_none_all, _, _ = cal_performance(list_dict_test_sample)

Accuracy is 49.17% with 0 none, 89 correct, and 92 wrong， total 181
Accuracy is 52.08% with 0 none, 163 correct, and 150 wrong， total 313
Accuracy is 51.01% with 0 none, 252 correct, and 242 wrong， total 494


In [19]:
print(count_none_MK, acc_MK, count_none_CA, acc_CA, count_none_all, acc_all, sep='\t')

0	0.4917	0	0.5208	0	0.5101


## End

In [42]:
sum_tokens_all = sum([ dict_test['usage']['total_tokens'] for dict_test in list_dict_test])
cost = sum_tokens_all / 1000 * 0.002 * 6.9348
print(f"{len(list_dict_test)} query and Cost: {cost:.2f} RMB")

494 query and Cost: 1.32 RMB


In [43]:
sum_tokens_prompt = sum([ dict_test['usage']['prompt_tokens'] for dict_test in list_dict_test])
sum_tokens_completion = sum([ dict_test['usage']['completion_tokens'] for dict_test in list_dict_test])
sum_tokens_prompt, sum_tokens_prompt/494, sum_tokens_completion, sum_tokens_completion/494

(92773, 187.7995951417004, 2749, 5.564777327935222)